In [1]:
from __future__ import absolute_import, division, unicode_literals
import sys
import io
import numpy as np
import logging
import argparse
import torch
import random
from transformers import *
import utils
import json
import pandas as pd
from tqdm import tqdm


/root/anaconda3/envs/SBERT-WK/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sbert import sbert

loading configuration file https://huggingface.co/binwang/bert-base-nli-stsb/resolve/main/config.json from cache at ./cache/741df96bc03932e0eade8d5031035bca0d2958179d96a37e973a5e1863450ccb.c5a70c2b03d4ec41c3610c8a99080e27c87d4599b28c5857597c193fc608bdfe
Model config BertConfig {
  "_name_or_path": "binwang/bert-base-nli-stsb",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/binwan

to device


In [5]:
sbert(['A girl', 'A woman'])

0.8690853

In [3]:
classifier = pipeline("text-classification", model = "microsoft/deberta-xlarge-mnli",device=2)

loading configuration file https://huggingface.co/microsoft/deberta-xlarge-mnli/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1c2adfd60f19f0beaf52e3c7988db42be627c0e366303902fec8997eed74b95f.4b3a81a93b125c8a71c26ca387b3068c017005c54e607aa7dd0b49d47862e204
Model config DebertaConfig {
  "_name_or_path": "microsoft/deberta-xlarge-mnli",
  "architectures": [
    "DebertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "CONTRADICTION",
    "1": "NEUTRAL",
    "2": "ENTAILMENT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "CONTRADICTION": 0,
    "ENTAILMENT": 2,
    "NEUTRAL": 1
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 48,
  "pad_token_id": 0,
  "pooler_dropout": 0,
 

In [4]:
import itertools
# Get max sim between 1 sentence and others
def get_max_sim(sentence, sentences):
    max_sim = 0
    for pair in itertools.product(sentence,sentences):
        sim = sbert([pair[0],pair[1]])
        if sim > max_sim:
            max_sim = sim
    return max_sim

In [5]:
import itertools
# Get max sim between 1 sentence and others
def get_mean_sim(sentence, sentences):
    mean_sim = []
    for pair in itertools.product(sentence,sentences):
        sim = sbert([pair[0],pair[1]])
        mean_sim.append(sim)
    return np.mean(mean_sim)

In [20]:
# df = pd.read_json('/root/thesis/ViLT/cosmos/test_data.json',orient="records", lines=True)

In [11]:
df = pd.read_csv('new_data.csv')

In [7]:
test_data = list(
    map(json.loads, open('/root/thesis/new_dataset/cosmos_anns_acm/acm_anns/test_data_2.json').readlines())
)

In [8]:
dataframe = pd.DataFrame(
    test_data, columns=["img_local_path", "caption1_modified","caption2_modified","context_label","maskrcnn_bboxes","bert_base_score"],
)

In [9]:
df = dataframe

In [12]:
tqdm.pandas()
df['nli_label'] = df.progress_apply(lambda x: classifier(x.caption1+x.caption2)[0], axis=1)

  1%|          | 11/1700 [00:01<02:52,  9.79it/s]/root/anaconda3/envs/SBERT-WK/lib/python3.7/site-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
100%|██████████| 1700/1700 [02:49<00:00, 10.00it/s]


In [13]:
df['sbert_wk_score'] = df['bert_base_score']

In [31]:
df['caption_3'] = df['caption_3'].apply( lambda x: x.strip("[]").replace("'","").split(", "))

In [34]:
tqdm.pandas()
df['sbert_wk_score_1_3'] = df.progress_apply(lambda x: 
get_max_sim([x.caption1_modified],x.caption_3)
, axis=1)
df['sbert_wk_score_2_3'] = df.progress_apply(lambda x: 
get_max_sim([x.caption2_modified],x.caption_3)
, axis=1)

100%|██████████| 1700/1700 [01:38<00:00, 17.27it/s]


In [ ]:
def get_false_scores(row):
    new_scores = [0,0,0,0]
    c1_fake = row['caption1'][:-1] + ' is not genuine.'
    new_scores[0] = sbert([c1_fake,row['caption2']])

    c1_fake = row['caption1'][:-1] + ' is fake.'
    new_scores[1] = sbert([c1_fake,row['caption2']])

    c1_fake = row['caption1'][:-1] + ' wasn\'t true.'
    new_scores[2] = sbert([c1_fake,row['caption2']])

    # c1_fake = 'No, ' + row['caption1'][:-1]
    # new_scores[3] = sbert([c1_fake,row['caption2']])
    return new_scores
df['false_scores'] = df.progress_apply(lambda x: get_false_scores(x), axis=1)

In [37]:
df['bb']= pd.read_csv('/root/thesis/COSMOS/pred_contexts.txt', header=None)[0]
# df['bb']= pd.read_csv('/root/thesis/COSMOS/pred_contexts_test_2.txt', header=None)[0]

In [72]:
df['a']=df['sbert_wk_score']
df['b']=df.progress_apply(lambda x: sbert([x.caption1.rstrip('.') + ' was fake.',x.caption2]),axis=1)
df['c']=df.progress_apply(lambda x: sbert([x.caption1,x.caption2.rstrip('.') + ' was fake.']),axis=1)

100%|██████████| 1700/1700 [00:33<00:00, 51.18it/s]


In [102]:
df['b1']=df.progress_apply(lambda x: sbert([x.caption1.rstrip('.') + ' is not genuine.',x.caption2]),axis=1)
df['c1']=df.progress_apply(lambda x: sbert([x.caption1,x.caption2.rstrip('.') + ' is not genuine.']),axis=1)

100%|██████████| 1700/1700 [00:32<00:00, 52.11it/s]


In [45]:
keywords = "fake, hoax, fabrication, supposedly, falsification, propaganda, deflection, deception, contradicted, defamation, lie, misleading, deceive, fraud, concocted, bluffing, made up, double meaning, alternative facts, trick, half-truth, untruth, falsehoods, inaccurate, disinformation, misconception"
df['f1']=df.progress_apply(lambda x: sbert([x.caption1,keywords]),axis=1)
df['f2']=df.progress_apply(lambda x: sbert([x.caption2,keywords]),axis=1)

100%|██████████| 1700/1700 [01:02<00:00, 27.08it/s]


In [72]:
def predict(row):
    if row['nli_label']['label'] == 'CONTRADICTION':
        return [True,'CONT']
    if row['nli_label']['label'] == 'ENTAILMENT':
        return [False,'ENTAIL']
    # false_scores = row['false_scores']
    # if false_scores[0] > row['sbert_wk_score']+0.05:
    #     return True
    # if false_scores[1] > row['sbert_wk_score']+0.05:
    #     return True
    # if false_scores[2] > row['sbert_wk_score']+0.05:
    #     return True
    # if false_scores[3] > row['sbert_wk_score']+0.05:
    #     return True

    # if row['a']+0.1<row['b'] or row['a']+0.1<row['c']:
    #     return True
    # if row['a']+0.1<row['b1'] or row['a']+0.1<row['c1']:
    #     return True
    if (row['f1'] < 0.1 and row['f2'] > 0.15) or (row['f2'] < 0.1 and row['f1'] > 0.15):
        return [True,'FAKE']
        
    if row['sbert_wk_score']>=0.5:
        
        # if row['sbert_wk_score']>0.7:
        #     if false_scores[0] > row['sbert_wk_score']+0.02:
        #         return True
        #     if false_scores[1] > row['sbert_wk_score']+0.02:
        #         return True
        #     if false_scores[2] > row['sbert_wk_score']+0.02:
        #         return True
        #     if false_scores[3] > row['sbert_wk_score']+0.02:
        #         return True
        # 52 cau similar, khong doi lap nhung OOC
        # if row['context_label']==1:
        #     print('g')
        return [False,'SBERT']
    
    # if (row['sbert_wk_score_1_3'] < 0.5 and row['sbert_wk_score_2_3'] > 0.5) \
    #     or (row['sbert_wk_score_1_3'] > 0.5 and row['sbert_wk_score_2_3'] < 0.5):
    #     # 18 cau khong doi lap, captiong match nhung OOC
    #     return False
    # 26 cau sai nhung no thay dung
    # 75 cau dung nhung no thay sai
    # Tong cau o day la 569. 17% cau o day sai
    return [row['bb'] > 0.5, 'iou']
    # if row['iou']<0.5:
    #     return False

In [42]:
# Language Baseline
df['predict'] =  (df['sbert_wk_score'] < 0.5)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

Actual       0    1
Predicted          
False      591  134
True       259  716
accuracy: 0.7688235294117647


In [73]:
# Language Baseline + NLI
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
df['method'] = df['predict'].apply(lambda x:x[1])
df['predict'] = df['predict'].apply(lambda x:x[0])
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

Actual       0    1
Predicted          
False      703   99
True       147  751
accuracy: 0.8552941176470589


In [57]:
df

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,...,bbs,bbs2,iou,nli_label,sbert_wk_score,bb,predict,f1,f2,method
0,test/0.jpg,Julian Castro at his announcement in San Anton...,Julian Castro at his announcement in San Anton...,0,https://www.nytimes.com/2019/06/13/us/politics...,"[[389.9706726074219, 72.9228744506836, 505.056...","PERSON at his announcement in GPE, GPE, on DAT...","[['Julian Castro', 'PERSON'], ['San Antonio', ...","PERSON at his announcement in GPE, GPE, on DATE.","[['Julian Castro', 'PERSON'], ['San Antonio', ...",...,"[[0.5392506122589111, 0.6175072193145752, 0.40...","[[0.5869272351264954, 0.6132014989852905, 0.33...",0.827233,"{'label': 'NEUTRAL', 'score': 0.979162871837616}",0.576995,0.145144,False,-0.078040,-0.060643,SBERT
1,test/1.jpg,Supporters of Tanzania's ruling Chama Cha Mapi...,A person sits on a truck as supporters of the ...,0,https://www.bbc.com/news/world-africa-54828934,"[[389.6280517578125, 8.949727058410645, 609.61...",Supporters of GPE's ruling ORG party come out ...,"[['Tanzania', 'GPE'], ['Chama Cha Mapinduzi', ...",A person sits on a truck as supporters of the ...,"[['Chama Cha Mapinduzi', 'PERSON'], ['Revoluti...",...,"[[0.418454110622406, 0.7030856609344482, 0.548...","[[0.490699827671051, 0.40026530623435974, 0.24...",0.500253,"{'label': 'NEUTRAL', 'score': 0.9272750020027161}",0.541939,1.000000,False,-0.087683,-0.087320,SBERT
2,test/2.jpg,"53,000 dead people turned up on the state’s vo...",These social media posts did not link to a rec...,1,https://www.snopes.com/fact-check/53000-dead-f...,"[[0.0, 14.214579582214355, 326.70501708984375,...",CARDINAL dead people turned up on the state’s ...,"[['53,000', 'CARDINAL'], ['November 2018', 'DA...",These social media posts did not link to a rec...,"[['Florida', 'GPE'], ['November 2018', 'DATE']...",...,"[[0.6021366119384766, 0.4899236559867859, 0.61...","[[0.5417162179946899, 0.3864124119281769, 0.58...",0.709888,"{'label': 'NEUTRAL', 'score': 0.8083608746528625}",0.234810,1.000000,True,-0.021542,0.110570,iou
3,test/3.jpg,"Actor, musician, director and devoted follower...",A shocking report about the former child actor...,1,https://www.snopes.com/fact-check/kirk-cameron...,"[[491.4454345703125, 361.714599609375, 538.475...","Actor, musician, director and devoted follower...","[['Christ', 'ORG'], ['Kirk Cameron', 'PERSON']]",A shocking report about the former child actor...,"[['Kirk Cameron', 'PERSON']]",...,"[[0.5500807762145996, 0.5195492506027222, 0.84...","[[0.45905983448028564, 0.570932149887085, 0.49...",0.576425,"{'label': 'CONTRADICTION', 'score': 0.84816104...",0.460771,1.000000,True,0.088550,0.407171,CONT
4,test/4.jpg,Men from the Maasai tribe perform a traditiona...,"And on the same day in Kenya's Narok county, y...",0,https://www.bbc.com/news/world-africa-56278224,"[[194.4141845703125, 31.10932731628418, 381.77...",Men from the GPE tribe perform a traditional j...,"[['Maasai', 'GPE'], ['Masai-Mara', 'ORG']]","And on DATE in GPE's GPE, young PERSON men tak...","[['the same day', 'DATE'], ['Kenya', 'GPE'], [...",...,"[[0.49438440799713135, 0.443972110748291, 0.93...","[[0.5386363863945007, 0.4829131066799164, 0.50...",0.598959,"{'label': 'NEUTRAL', 'score': 0.769950807094574}",0.619695,0.070260,False,-0.085142,-0.153572,SBERT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,test/1695.jpg,President Obama trademarked the name 'Obamacar...,There was no truth that Obama family millions ...,1,https://www.snopes.com/fact-check/obamacare-ro...,"[[283.2491455078125, 28.175857543945312, 700.4...",President PERSON trademarked the name 'ORG' an...,"[['Obama', 'PERSON'], ['Obamacare', 'ORG']]",There was no truth that PERSON family MONEY wi...,"[['Obama', 'PERSON'], ['millions of dollars', ...",...,"[[0.5283848643302917, 0.42085203528404236, 0.5...","[[0.43506768345832825, 0.5325986742973328, 0.4...",0.

In [57]:
# Language Baseline + NLI
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

Actual     False  True
Predicted             
False        441   699
True         155   918
accuracy: 0.7994117647058824


In [62]:
col=['img_local_path','caption1','caption2','context_label','caption1_modified',
'caption2_modified','caption_3','bb','predict','f1','f2','method']
df[col].to_csv('ana.csv',index=False)

In [61]:
df['bb'] = df['bb'].astype(int)
df['predict'] = df['predict'].astype(int)

In [16]:
# Language Baseline + neu phu dinh + image_caption threshold 0.5
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

Actual       0    1
Predicted          
False      621   75
True       229  775
accuracy: 0.8211764705882353


In [11]:
# Language Baseline + neu phu dinh + image_caption + cosmos
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)
#0.808

Actual       0    1
Predicted          
False      722  101
True       128  749
accuracy: 0.8652941176470588


In [31]:
# entailment + neu phu dinh
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)
#0.808

Actual       0    1
Predicted          
False      685   47
True       165  803
accuracy: 0.8752941176470588


In [39]:
## misleading words (is fake, is not genuine,...) => không bias
# entailment + neu phu dinh
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)
#0.808

Actual       0    1
Predicted          
False      707   64
True       143  786
accuracy: 0.8782352941176471


In [43]:
df['caption_3'].head(3)

0    [a man standing at a podium in front of a crow...
1    [man sitting on the top of an unmingy in front...
2    [a group of people standing around a table wit...
Name: caption_3, dtype: object